# Sprint 13 - Proyecto

### 1. Preparar los datos

##### Cargar las librerías

In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    f1_score,
    mean_squared_error
)

# Utilities
from sklearn.utils import shuffle


#### 1.1. Abre los archivos y examina los datos

In [3]:
#Cargar los archivos

gold_recovery_train = pd.read_csv('data/gold_recovery_train.csv')
gold_recovery_test = pd.read_csv('data/gold_recovery_test.csv')
gold_recovery_full = pd.read_csv('data/gold_recovery_full.csv')

In [4]:
#Análisis exploratorio de datos para los tres conjuntos:

# Para cada dataset
print("Dimensiones del datasets:")
print(f"Train: {gold_recovery_train.shape}")
print(f"Test: {gold_recovery_test.shape}") 
print(f"Full: {gold_recovery_full.shape}")
print()
print("Valores ausentes de los datasets:")
print(f"Train - Total valores ausentes: {gold_recovery_train.isnull().sum().sum()}, es decir, el {(gold_recovery_train.isnull().sum().sum()) / gold_recovery_train.size * 100} está ausente")
print(f"Test - Total valores ausentes: {gold_recovery_test.isnull().sum().sum()}, es decir, el {(gold_recovery_test.isnull().sum().sum()) / gold_recovery_test.size * 100} está ausente")
print(f"Full - Total valores ausentes: {gold_recovery_full.isnull().sum().sum()}, es decir, el {(gold_recovery_full.isnull().sum().sum()) / gold_recovery_full.size * 100} está ausente")

Dimensiones del datasets:
Train: (16860, 87)
Test: (5856, 53)
Full: (22716, 87)

Valores ausentes de los datasets:
Train - Total valores ausentes: 30320, es decir, el 2.0670566258982013 está ausente
Test - Total valores ausentes: 2360, es decir, el 0.7603876688318383 está ausente
Full - Total valores ausentes: 36587, es decir, el 1.851295253940207 está ausente


In [5]:
# Revisar qué columnas faltan en test vs train
train_cols = set(gold_recovery_train.columns)
test_cols = set(gold_recovery_test.columns)
missing_in_test = train_cols - test_cols

print(f"Columnas que están en train pero NO en test: {len(missing_in_test)}")
print("Columnas faltantes:")
print(list(missing_in_test)[:])

Columnas que están en train pero NO en test: 34
Columnas faltantes:
['primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_pb', 'secondary_cleaner.output.tail_au', 'secondary_cleaner.output.tail_sol', 'rougher.output.recovery', 'secondary_cleaner.output.tail_ag', 'final.output.concentrate_ag', 'rougher.output.tail_pb', 'rougher.output.tail_au', 'final.output.tail_au', 'final.output.concentrate_sol', 'primary_cleaner.output.tail_sol', 'primary_cleaner.output.concentrate_sol', 'rougher.calculation.floatbank11_sulfate_to_au_feed', 'final.output.tail_pb', 'rougher.calculation.floatbank10_sulfate_to_au_feed', 'primary_cleaner.output.concentrate_au', 'rougher.output.concentrate_ag', 'secondary_cleaner.output.tail_pb', 'rougher.calculation.sulfate_to_au_concentrate', 'rougher.output.tail_sol', 'final.output.concentrate_au', 'rougher.output.tail_ag', 'rougher.calculation.au_pb_ratio', 'final.output.concentrate_pb', 'rougher.output.concentrate_pb', 'primary_cleaner.output.concentrate_p

Se observa que las columnas faltantes en el dataset de prueba (test) probablemente sean las variables objetivo, lo cual explicaría el porqué se encuentran en el dataset de entrenamiento pero no en el de prueba.

##### 1.2. Comprueba que el cálculo de la recuperación sea correcto. Calcula la recuperación de la característica rougher.output.recovery mediante el conjunto de entrenamiento

In [6]:
# Identificar las columnas necesarias, aquellas relacionadas conrougher

rougher_cols = [col for col in gold_recovery_train.columns if 'rougher' in col]
print('Columnas rougher disponibles:')
for col in rougher_cols:
    print(f" - {col}")

Columnas rougher disponibles:
 - rougher.calculation.sulfate_to_au_concentrate
 - rougher.calculation.floatbank10_sulfate_to_au_feed
 - rougher.calculation.floatbank11_sulfate_to_au_feed
 - rougher.calculation.au_pb_ratio
 - rougher.input.feed_ag
 - rougher.input.feed_pb
 - rougher.input.feed_rate
 - rougher.input.feed_size
 - rougher.input.feed_sol
 - rougher.input.feed_au
 - rougher.input.floatbank10_sulfate
 - rougher.input.floatbank10_xanthate
 - rougher.input.floatbank11_sulfate
 - rougher.input.floatbank11_xanthate
 - rougher.output.concentrate_ag
 - rougher.output.concentrate_pb
 - rougher.output.concentrate_sol
 - rougher.output.concentrate_au
 - rougher.output.recovery
 - rougher.output.tail_ag
 - rougher.output.tail_pb
 - rougher.output.tail_sol
 - rougher.output.tail_au
 - rougher.state.floatbank10_a_air
 - rougher.state.floatbank10_a_level
 - rougher.state.floatbank10_b_air
 - rougher.state.floatbank10_b_level
 - rougher.state.floatbank10_c_air
 - rougher.state.floatbank10_

In [7]:
# Realizar el cálculo de la característica rougher.output.recovery
#C: concentración de la sustancia en el concentrado
C = gold_recovery_train['rougher.output.concentrate_au']

#F: concentración de la sustancia en la alimentación(feed)
F = gold_recovery_train['rougher.input.feed_au']

#T: concentración de la sustancia en las colas (tails)
T = gold_recovery_train['rougher.output.tail_au']

#Calcular la recuperación
recovery_calculated = (C * (F - T)/(F * (C - T))) * 100
print(f"Recovery calculada: {recovery_calculated.head().values}")

Recovery calculada: [87.10776293 86.84326051 86.84230826 87.22642953 86.68879414]


In [13]:
# Comparar los valores calculados y los valores originales

tolerance = 0.01

close_match = abs(recovery_calculated - gold_recovery_train['rougher.output.recovery']) < tolerance
print(f'Valores similares (±{tolerance}): {close_match.sum()} de {len(close_match)}')

# Calcular el MAE
mae = abs(recovery_calculated - gold_recovery_train['rougher.output.recovery']).mean()
print(f"Error Absoluto Medio: {mae}")

Valores similares (±0.01): 14287 de 16860
Error Absoluto Medio: 9.303415616264301e-15


De acuerdo con los datos calculados, se observa que 14287 datos de 16860 (aproximadamente el 84.7%) se encuentran dentro de un intervalo del ±10%, dejando algunos datos fuera por posibles causas como la forma en que se tomaron los datos, la precisión del análisis, la precisión de los instrumentos, incertidumbre de los instrumentos, error al tomar las medidas, etc.

Por otro lado, se observa que el Error Absoluto Medio es un valor muy pequeño, lo cual aporta información sobre que los datos calculados y los datos originales se encuentran muy cercanos entre ellos. 